# Geometric Program: Circuit Sizing

This notebook demonstrates how to use CVXPYlayers with **geometric programs** (GPs).

Geometric programs are a class of optimization problems where the objective and constraints are formed from *posynomials* — sums of monomials with positive coefficients. CVXPY supports GPs via `cp.Variable(pos=True)` and the `gp=True` flag.

**Problem**: We size transistor widths to minimize propagation delay subject to area and power budgets. This is a classic GP from Boyd et al. (2007).

**Key API**:
- `cp.Variable(pos=True)` — positive variables for GP
- `problem.is_dgp(dpp=True)` — verify DGP-DPP compliance
- `CvxpyLayer(..., gp=True)` — enable GP mode

In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import torch

from cvxpylayers.torch import CvxpyLayer

## Define the Geometric Program

We model a simple 3-stage digital circuit. Each stage $i$ has a transistor width $w_i > 0$.

- **Delay** of each stage is inversely proportional to its width: $d_i = \alpha_i / w_i$
- **Area** of each stage is proportional to its width: total area $= \sum_i w_i$
- **Power** scales as width times switching frequency: total power $= \sum_i \beta_i w_i$

The total delay is $\sum_i d_i = \sum_i \alpha_i / w_i$, which we minimize subject to parametric area and power budgets.

In [ ]:
n_stages = 3

# Positive variables (required for GP)
w = cp.Variable(n_stages, pos=True, name="widths")

# Fixed coefficients
alpha = np.array([1.0, 1.5, 0.8])  # delay coefficients
beta = np.array([0.5, 0.8, 0.6])   # power coefficients

# Parametric budgets
A_max = cp.Parameter(pos=True, name="area_budget")
P_max = cp.Parameter(pos=True, name="power_budget")

# Objective: minimize total delay (sum of alpha_i / w_i)
delay = sum(alpha[i] * cp.inv_pos(w[i]) for i in range(n_stages))
objective = cp.Minimize(delay)

# Constraints
constraints = [
    cp.sum(w) <= A_max,                        # area budget
    beta @ w <= P_max,                          # power budget
    w >= 0.1,                                   # minimum width
]

problem = cp.Problem(objective, constraints)
assert problem.is_dgp(dpp=True), "Problem must be DGP-DPP compliant"
print(f"Problem is DGP: {problem.is_dgp()}, DPP: {problem.is_dgp(dpp=True)}")

## Create the CvxpyLayer with `gp=True`

When `gp=True`, CVXPYlayers internally transforms the GP to an equivalent convex problem in log-space, solves it, and maps the solution back. Gradients flow through this transformation automatically.

In [ ]:
layer = CvxpyLayer(problem, parameters=[A_max, P_max], variables=[w], gp=True)

# Solve for specific budget values
A_max_tch = torch.tensor(5.0, dtype=torch.float64, requires_grad=True)
P_max_tch = torch.tensor(3.0, dtype=torch.float64, requires_grad=True)

(w_opt,) = layer(A_max_tch, P_max_tch)
print(f"Optimal widths: {w_opt.detach().numpy().round(4)}")
print(f"Total delay:    {sum(alpha / w_opt.detach().numpy()):.4f}")
print(f"Total area:     {w_opt.sum().item():.4f} (budget: {A_max_tch.item()})")
print(f"Total power:    {(torch.tensor(beta) @ w_opt).item():.4f} (budget: {P_max_tch.item()})")

## Sensitivity Analysis via Gradients

The key advantage of differentiable optimization: we can compute how the optimal delay changes with respect to the budget parameters. This gives us *sensitivity* or *shadow price* information.

In [ ]:
# Compute gradient of total delay w.r.t. budgets
total_delay = sum(torch.tensor(alpha[i]) * (1.0 / w_opt[i]) for i in range(n_stages))
total_delay.backward()

print(f"d(delay)/d(area_budget)  = {A_max_tch.grad.item():.6f}")
print(f"d(delay)/d(power_budget) = {P_max_tch.grad.item():.6f}")
print()
print("Negative gradients mean increasing the budget decreases delay.")
print(f"A unit increase in area budget reduces delay by ~{-A_max_tch.grad.item():.4f}")
print(f"A unit increase in power budget reduces delay by ~{-P_max_tch.grad.item():.4f}")

## Delay vs. Area Budget Tradeoff

Sweep the area budget and plot how the optimal delay changes. The gradient gives the local slope of this curve.

In [ ]:
area_values = np.linspace(1.5, 10.0, 30)
delays = []
sensitivities = []
P_fixed = torch.tensor(5.0, dtype=torch.float64)

for a_val in area_values:
    A_tch = torch.tensor(a_val, dtype=torch.float64, requires_grad=True)
    (w_sol,) = layer(A_tch, P_fixed)
    d = sum(torch.tensor(alpha[i]) * (1.0 / w_sol[i]) for i in range(n_stages))
    d.backward()
    delays.append(d.item())
    sensitivities.append(A_tch.grad.item())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(area_values, delays, 'b-', linewidth=2)
ax1.set_xlabel('Area Budget')
ax1.set_ylabel('Optimal Delay')
ax1.set_title('Delay vs. Area Budget')
ax1.grid(True, alpha=0.3)

ax2.plot(area_values, sensitivities, 'r-', linewidth=2)
ax2.set_xlabel('Area Budget')
ax2.set_ylabel('d(delay)/d(area_budget)')
ax2.set_title('Sensitivity (Shadow Price)')
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("The sensitivity approaches zero as the area budget becomes non-binding.")